In [ ]:
import numpy as np

Import phi and phi_test from train and test datasets using NumPy's loadtxt function

In [ ]:
phi = np.loadtxt('train.csv', dtype='float', delimiter=',', skiprows=1,
                 usecols=tuple(range(1, 14)))

In [ ]:
phi

array([[1.06411199e-02, 0.00000000e+00, 2.81524927e-01, ...,
        7.71748449e-01, 4.26074896e-01, 1.00000000e+00],
       [2.21317669e-04, 2.94736842e-01, 5.34457478e-01, ...,
        9.98562711e-01, 1.19001387e-01, 1.00000000e+00],
       [1.36513324e-02, 0.00000000e+00, 7.00879765e-01, ...,
        9.15603409e-01, 7.40638003e-02, 1.00000000e+00],
       ...,
       [7.47001646e-02, 0.00000000e+00, 6.46627566e-01, ...,
        1.00000000e+00, 3.34812760e-01, 1.00000000e+00],
       [1.37252923e-03, 0.00000000e+00, 2.96920821e-01, ...,
        9.76776439e-01, 3.37586685e-01, 1.00000000e+00],
       [6.24949841e-04, 7.36842105e-01, 6.52492669e-02, ...,
        9.27732110e-01, 8.46047157e-02, 1.00000000e+00]])

In [ ]:
phi_test = np.loadtxt('test.csv', dtype='float', delimiter=',',
                      skiprows=1, usecols=tuple(range(1, 14)))

In [ ]:
phi_test

array([[1.09096460e-03, 3.15789474e-01, 1.63856305e-01, ...,
        9.94250845e-01, 2.90707351e-01, 1.00000000e+00],
       [3.73205208e-03, 0.00000000e+00, 2.53665689e-01, ...,
        1.00000000e+00, 1.16504854e-01, 1.00000000e+00],
       [1.37557979e-01, 0.00000000e+00, 6.46627566e-01, ...,
        6.13495386e-02, 3.81969487e-01, 1.00000000e+00],
       ...,
       [5.00634279e-04, 2.21052632e-01, 1.89882698e-01, ...,
        1.00000000e+00, 9.32038835e-02, 1.00000000e+00],
       [1.07489125e-03, 4.21052632e-01, 2.18108504e-01, ...,
        9.80710071e-01, 1.14563107e-01, 1.00000000e+00],
       [5.11066221e-02, 0.00000000e+00, 6.46627566e-01, ...,
        8.93590196e-01, 1.44244105e-01, 1.00000000e+00]])

Import y from train dataset using the loadtxt function

In [ ]:
y = np.loadtxt('train.csv', dtype='float', delimiter=',', skiprows=1,
               usecols=14, ndmin=2)

Concatenate coloumn of 1s to right of phi and phi_test

In [ ]:
phi_test = np.concatenate((phi_test, np.ones((105, 1))), axis=1)
phi = np.concatenate((phi, np.ones((400, 1))), axis=1)

Apply min max scaling on each coloumn of phi and phi_test

In [ ]:
for i in range(13):
    col_max = max(phi[:, i])
    col_min = min(phi[:, i])
    phi[:, i] = (phi[:, i] - col_min) / (col_max - col_min)
    phi_test[:, i] = (phi_test[:, i] - col_min) / (col_max - col_min)

Apply log scaling on y

In [ ]:
y = np.log(y)

Define a function to calculate change in error function based on phi, w and p norm

In [ ]:
def delta_w(p, phi, w):
    if p == 2:
        deltaw = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambd * p * np.power(np.absolute(w), (p - 1)))
    if p < 2 and p > 1:
        deltaw = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambd * p * np.power(np.absolute(w), (p - 1)) * np.sign(w))
    return deltaw

Make a dictionary containing filenames as keys and p as values

In [ ]:
filenames = {'output.csv': 2.0,
             'output_p1.csv': 1.75,
             'output_p2.csv': 1.5,
             'output_p3.csv': 1.3
             }

For each item in this dictionary:

Set the w to all 0s

In [ ]:
for (fname, p) in filenames.items():
    # Set initial w to zeros
    w = np.zeros((14, 1))

    # Hyperparameter lambda value
    lambd = 0.2

    # Maximum step size
    t = 0.00012

    # Calculate new value of w
    w_new = w - t * delta_w(p, phi, w)

    i = 0
    # Repeat steps until error between consecutive w is less than threshold
    while(np.linalg.norm(w_new-w) > 10 ** -10):
        w = w_new
        w_new = w - t * delta_w(p, phi, w)
        i = i + 1

In [ ]:
# Load values of id
id_test = np.loadtxt('test.csv', dtype='int', delimiter=',',
                         skiprows=1, usecols=0, ndmin=2)

    # Calculate y for test data
y_test = np.exp(np.dot(phi_test, w_new))

    # Save the ids and y
np.savetxt(fname, np.concatenate((id_test, y_test), axis=1),
               delimiter=',', fmt=['%d', '%f'], header='ID,MEDV', comments='')